In [37]:
import soundfile as sf
import scipy.signal
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random

In [2]:
data, fs = sf.read("dolphinsound.wav")
ch1 = data[:,0]
ch1.shape

(40320000,)

In [109]:
f,t,z = scipy.signal.stft(ch1, fs=fs, window='hann', nperseg=1024, 
                      noverlap=int(1024*.9), nfft=None, boundary=None,
                      padded=False
)

In [4]:
w = scipy.signal.get_window('hann', 1024, fftbins=True)

In [5]:
s = ch1[:1024]

In [6]:
X = scipy.fft.fft(s*w)*2/1024

In [7]:
np.sum(X[:513] - z[:,0])

0j

In [88]:
df = pd.read_csv("whistle_time.csv", sep=r'\s*,\s*')

<ipython-input-88-ae1cf25b8554>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("whistle_time.csv", sep=r'\s*,\s*')


In [126]:
process = np.zeros(515).reshape(515,1)
for index, data in df.iterrows():
    random.seed(index)
    start = int(math.ceil((data["start"]-t[0])/time_shift*96000))
    end = int(math.floor((data["end"]-t[0])/time_shift*96000))
    choose = random.sample(range(start, end+1), k=20)
    X = z[:,choose]
    X = abs(X)
    target = (np.ones(20)*data["target"]).reshape(1,20)
    i = (np.ones(20)*index).reshape(1,20)
    X = np.concatenate((X, target, i), axis=0)
    process = np.concatenate((process, X), axis=1)
process.shape

(515, 1201)

In [127]:
n = np.delete(process.T, 0, axis=0)
col = ["feature {}".format(i) for i in range(1,514)]
col.append("target")
col.append("id(in whistle_time.csv file)")
df_create = pd.DataFrame(n, columns=col)
df_create.head()

,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10,...,feature 506,feature 507,feature 508,feature 509,feature 510,feature 511,feature 512,feature 513,target,id(in whistle_time.csv file)
0,0.002335,0.001528,0.000376,0.000277,0.000266,0.000333,0.000568,0.000363,0.000654,0.000657,...,0.000019,0.000011,0.000008,0.000032,0.000043,0.000034,0.000034,0.000035,1.0,0.0
1,0.001655,0.000828,0.000371,0.000259,0.000084,0.000225,0.000252,0.000195,0.000338,0.000440,...,0.000033,0.000051,0.000040,0.000053,0.000039,0.000018,0.000032,0.000053,1.0,0.0
2,0.000401,0.000588,0.000240,0.000400,0.000231,0.000103,0.000306,0.000377,0.000214,0.000234,...,0.000031,0.000016,0.000016,0.000022,0.000011,0.000013,0.000012,0.000004,1.0,0.0
3,0.000997,0.000663,0.000363,0.001196,0.000887,0.000140,0.000559,0.000732,0.000419,0.001036,...,0.000046,0.000031,0.000026,0.000053,0.000036,0.000009,0.000013,0.000024,1.0,0.0
4,0.001638,0.001067,0.000291,0.000091,0.000134,0.000125,0.000531,0.000638,0.000332,0.000423,...,0.000027,0.000009,0.000035,0.000022,0.000022,0.000020,0.000058,0.000036,1.0,0.0


In [131]:
df_create.to_pickle("dol_feature")

In [132]:
df1 = pd.read_pickle("dol_feature")
df1.head()

,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10,...,feature 506,feature 507,feature 508,feature 509,feature 510,feature 511,feature 512,feature 513,target,id(in whistle_time.csv file)
0,0.002335,0.001528,0.000376,0.000277,0.000266,0.000333,0.000568,0.000363,0.000654,0.000657,...,0.000019,0.000011,0.000008,0.000032,0.000043,0.000034,0.000034,0.000035,1.0,0.0
1,0.001655,0.000828,0.000371,0.000259,0.000084,0.000225,0.000252,0.000195,0.000338,0.000440,...,0.000033,0.000051,0.000040,0.000053,0.000039,0.000018,0.000032,0.000053,1.0,0.0
2,0.000401,0.000588,0.000240,0.000400,0.000231,0.000103,0.000306,0.000377,0.000214,0.000234,...,0.000031,0.000016,0.000016,0.000022,0.000011,0.000013,0.000012,0.000004,1.0,0.0
3,0.000997,0.000663,0.000363,0.001196,0.000887,0.000140,0.000559,0.000732,0.000419,0.001036,...,0.000046,0.000031,0.000026,0.000053,0.000036,0.000009,0.000013,0.000024,1.0,0.0
4,0.001638,0.001067,0.000291,0.000091,0.000134,0.000125,0.000531,0.000638,0.000332,0.000423,...,0.000027,0.000009,0.000035,0.000022,0.000022,0.000020,0.000058,0.000036,1.0,0.0
